# Model builder
## Life Cycle Assessment of ``{Operational Energy Moerschberg for 1 year: 1}``
###[CONTEXT]
The model relies on datapackages to ensure replicability of the calculation.


In [212]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import bw_processing as bwp
import numpy as np
from pathlib import Path
import pandas as pd


In [213]:
bd.projects.delete_project('ei38-teaching-25')

'default'

In [214]:
bi.restore_project_directory("/srv/data/projects/ecoinvent38-25.tar.gz")
bd.projects.set_current('ei38-teaching-25')


Restoring project backup archive - this could take a few minutes...


In [215]:
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	ei 3.8 cutoff

In [216]:
# bd.projects.migrate_project_25()

In [217]:
if "foreground" in bd.databases:
    del bd.databases["foreground"]
    
foreground_importer = bi.ExcelImporter("./data/inputs/lci_moeschberg.xlsx")
foreground_importer.apply_strategies()
foreground_importer.match_database("biosphere3", fields=("name", "unit", "categories"))
foreground_importer.match_database("ei 3.8 cutoff", fields=("name", "unit", "location"))
foreground_importer.statistics()
foreground_importer.write_database()

Extracted 1 worksheets in 0.00 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 6.83 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
5 datasets
18 exchanges
0 unlinked exchanges
  
Title: Writing activities to SQLite3 da

In [218]:
bd.databases

Databases dictionary with 3 object(s):
	biosphere3
	ei 3.8 cutoff
	energy_moeschberg

In [219]:
fu = bd.get_activity(database="energy_moeschberg", name = "energy demand, operational, Hotel Moeschberg")
fu

'energy demand, operational, Hotel Moeschberg' (unit, CH, None)

In [220]:
heat = bd.get_activity(database="energy_moeschberg", name = "heat supply, Hotel Moeschberg, 2021")

In [221]:
wood_pellets_heat = bd.get_node(database="ei 3.8 cutoff", name='wood pellets, burned in stirling heat and power co-generation unit, 3kW electrical, future', unit='megajoule')

In [222]:
wood_pellets_market = bd.get_node(database="ei 3.8 cutoff", name = "market for wood pellet, measured as dry mass", location = "RER")

In [223]:
wood_pellets_prod_RER = bd.get_node(database="ei 3.8 cutoff", name = "wood pellet production", location = "RER")

In [224]:
if "Regionalized_db" in bd.databases:
    del bd.databases["Regionalized_db"]

In [225]:
Regionalized_db = bd.Database("Regionalized_db", backend = "iotable") #need iotable to be able to do the regionalization

In [226]:
Regionalized_db.write({})

In [227]:
bd.databases

Databases dictionary with 4 object(s):
	Regionalized_db
	biosphere3
	ei 3.8 cutoff
	energy_moeschberg

In [228]:
foreground=bd.Database('energy_moeschberg')

In [229]:
regionalized_fu=Regionalized_db.new_node(code="new-fu", name="energy demand, operational, Hotel Moeschberg - regionalized", location="CH", unit="unit")
regionalized_fu.save()

In [230]:
regionalized_heat=Regionalized_db.new_node(code="new-heat", name='heat supply, Hotel Moeschberg, 2021 - regionalized', location="CH", unit="megajoule")
regionalized_heat.save()

In [231]:
new_wood_pellets_heat = Regionalized_db.new_node(code="pellets-heat", name="wood pellets, burned in stirling heat and power co-generation unit, 3kW electrical, future - regionalized", location="CH", unit="megajoule")
new_wood_pellets_heat.save()

In [232]:
new_wood_pellets_market = Regionalized_db.new_node(code="pellets-market", name="market for wood pellet, measured as dry mass", location="CH", unit="kilogram")
new_wood_pellets_market.save()

In [233]:
new_pellets_prod = Regionalized_db.new_node(code="pellets-prod", name="wood pellet production", location="CH", unit="kilogram")
new_pellets_prod.save()

In [234]:
from collections import defaultdict

new_pellets_tech = defaultdict(float)

for exc in wood_pellets_prod_RER.technosphere():
    new_pellets_tech[bd.get_node(
        database = "ei 3.8 cutoff",
        name = exc.input['name'],
        location = exc.input['location']
    )] += exc['amount']

In [235]:
tech_edges = [
    {"row": regionalized_fu.id, "col": regionalized_fu.id, "amount":1},
    {"row": fu.id, "col": regionalized_fu.id, "amount":1, "flip":True}, #consume the old activity to copy exchanges
    {"row": heat.id, "col": regionalized_fu.id, "amount":655489 , "flip":False},
    {"row": regionalized_heat.id, "col": regionalized_fu.id, "amount":655489, "flip":True},
] + [
    {"row": regionalized_heat.id, "col": regionalized_heat.id, "amount":1},
    {"row": heat.id, "col": regionalized_heat.id, "amount":1, "flip":True}, #consume the old activity to copy exchanges
    {"row": wood_pellets_heat.id, "col": regionalized_heat.id, "amount":0.9953, "flip":False},
    {"row": new_wood_pellets_heat.id, "col": regionalized_heat.id, "amount":0.9953, "flip":True},
] + [
    {"row": new_wood_pellets_heat.id, "col": new_wood_pellets_heat.id, "amount":1},
    {"row": wood_pellets_heat.id, "col": new_wood_pellets_heat.id, "amount":1, "flip":True}, #consume the old activity to copy exchanges
    {"row": new_wood_pellets_market.id, "col": new_wood_pellets_heat.id, "amount":0.0229911254795743, "flip":True},
    {"row": wood_pellets_market.id, "col": new_wood_pellets_heat.id, "amount": 0.0229911254795743},
] + [
    {"row": new_wood_pellets_market.id, "col": new_wood_pellets_market.id, "amount":1},
    {"row": wood_pellets_market.id, "col": new_wood_pellets_market.id, "amount":1, "flip":True},
    {"row": new_pellets_prod.id, "col": new_wood_pellets_market.id, "amount": 1, "flip":True},
    {"row": wood_pellets_prod_RER.id, "col": new_wood_pellets_market.id, "amount": 1},
] + [
    {"row": new_pellets_prod.id, "col": new_pellets_prod.id, "amount": 1},
    {"row": wood_pellets_prod_RER.id, "col": new_pellets_prod.id, "amount": 1, "flip":True},
    {
        "row": bd.get_activity(
            database="ei 3.8 cutoff", 
            name='market group for electricity, medium voltage',
            location='RER'
        ).id, 
        "col": new_pellets_prod.id, 
        "amount": 0.096
    },
] + [
    {"row": key.id, "col": new_pellets_prod.id, "amount": value}
    for key, value in new_pellets_tech.items() if key['location']=="Europe without Switzerland"
] + [
    {
        "row": bd.get_activity(
            database="ei 3.8 cutoff", 
            name='market for electricity, medium voltage',
            location='CH'
        ).id, 
        "col": new_pellets_prod.id, 
        "amount": 0.096,
        "flip":True
    },
] + [
    {"row": bd.get_activity(database="ei 3.8 cutoff", name=key['name'], location="CH").id, 
      "col": new_pellets_prod.id, "amount": value, "flip":True}
    for key, value in new_pellets_tech.items() if key['location']=="Europe without Switzerland"

]

In [236]:
Regionalized_db.write_exchanges(tech_edges, [], ["ei 3.8 cutoff"])

Starting IO table write
Adding technosphere matrix
Adding biosphere matrix
Finalizing serialization


In [237]:
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')

In [238]:
fu.new_edge(input=regionalized_fu, amount=0, type="technosphere").save()

In [239]:
_, data_objs, _ = bd.prepare_lca_inputs({fu: 1}, ipcc)

In [240]:
lca = bc.LCA({fu.id: 1}, data_objs=data_objs)
lca.lci()
lca.lcia()
lca.score

4288.388560965337

In [242]:
lca.lcia({regionalized_fu.id: 1})
lca.score

3629.403444136348

In [118]:
bd.databases

Databases dictionary with 4 object(s):
	Regionalized_db
	biosphere3
	ei 3.8 cutoff
	energy_moeschberg

In [122]:
Regionalized_db.write_exchanges(tech_edges, [], ["ei 3.8 cutoff"])

Starting IO table write
Adding technosphere matrix
Adding biosphere matrix
Finalizing serialization


In [123]:
Regionalized_db.datapackage

'energy demand, operational, Hotel Moeschberg' (unit, CH, None)

In [124]:
fu.new_edge(input=regionalized_fu, amount=0, type="technosphere").save()

In [125]:
_, data_objs, _ = bd.prepare_lca_inputs({fu:1}, ipcc)

In [126]:
lca = bc.LCA({fu.id: 1}, data_objs=data_objs)
lca.lci()
lca.lcia()
lca.score

NonsquareTechnosphere: Technosphere matrix is not square: 19571 activities (columns) and 19572 products (rows). Use LeastSquaresLCA to solve this system, or fix the input data

In [ ]:
for obj in objects:
    print(obj.metadata["name"])

In [ ]:
data, _ = objects[2].get_resource("energy_moeschberg_technosphere_matrix.data")
indices, _ = objects[2].get_resource("energy_moeschberg_technosphere_matrix.indices")
flip, _ = objects[2].get_resource("energy_moeschberg_technosphere_matrix.flip")
unique_indices = set([a for b in indices for a in b])
mapping_act=dict.fromkeys(int(i) for i in unique_indices) #json accepts only int not int32
for i in unique_indices:
    mapping_act[i]=bd.get_activity(i)['name']
rows = [tup[0] for tup in indices]
cols = [tup[1] for tup in indices]

In [ ]:
df = pd.DataFrame(columns=['id','name','location','reference product'],index=np.arange(len(unique_indices)))
n=0
for i in unique_indices: 
    df['id'][n]=(i)
    df['name'][n]=(bd.get_activity(i)['name'])
    df['location'][n]=(bd.get_activity(i)['location'])
    df['reference product'][n]=(bd.get_activity(i)['reference product'])
    n += 1

In [ ]:
df

In [ ]:
bd.get_activity(database = bg_19,
                name="electricity production, hydro, run-of-river",
                location="CH", 
                product = "electricity, high voltage").id

In [ ]:
import pandas as pd

#initialize a dataframe
df = pd.DataFrame(
	[[21, 'Amol', 72, 67],
	[23, 'Lini', 78, 69],
	[32, 'Kiku', 74, 56],
	[52, 'Ajit', 54, 76]],
	columns=['rollno', 'name', 'physics', 'botony'])

print('DataFrame with default index\n', df)

#set multiple columns as index
df = df.set_index(['rollno','name'])

print('\nDataFrame with MultiIndex\n',df)

In [ ]:
import pandas as pd 
id_all = []


for idx, row in df.iterrows():
    act_name = row["name"]
    act_location = row["location"]
    act_ref = row["reference product"]
    try:
        act_bg_base_id = bd.get_activity(database = bg_base,name=act_name,location=act_location, product = act_ref).id
    except:
        act_bg_base_id =row["id"]
            
    try:
        act_bg_19_id = bd.get_activity(database = bg_19,name=act_name,location=act_location, product = act_ref).id
    except:
        act_bg_19_id =row["id"]

            
            
    id_all.append({
        "name": row["name"],
        "id_ei38":row["id"],
        "id_pre_base":act_bg_base_id,
        "id_pre_19":act_bg_19_id,
                  })
scenarios_ids=pd.DataFrame(id_all)    
scenarios_ids = scenarios_ids.set_index("id_ei38", drop=False)
scenarios_ids

In [ ]:

    #     for row in col[1]:
    #         print(row)
    #         # print(col.loc[1])

In [ ]:
import pandas as pd
matrix_raw = pd.DataFrame({"row":rows, 
                           "col":cols, 
                           "from":[mapping_act[idx] for idx in rows], 
                           "to":[mapping_act[idx] for idx in cols],
                           "data":data
                          })
matrix = matrix_raw.pivot(index='row',columns='col', values='data').fillna(0)
# mapping
# matrix_raw

In [ ]:
# objects[2].metadata["name"]
# dp_dict = dict(
#         zip(indices.astype("object"),[[tup[0], tup[1]] for tup in zip(data,flip)])
#                 )
# dp_dict

In [ ]:
new_data = np.array([val[0] for val in dp_dict.items()])

In [ ]:
mapping_act

In [ ]:
for col in scenarios_ids.iteritems():
    scenario_name = col[0]
    if "name" not in scenario_name: 
        for idx, row in scenarios_ids.iterrows():
            original_id = idx
            new_id = row[scenario_name]
            new_indices = {
                    old:new
            }

new_arrays = {
    (213912,213913): [655489,655489*0.5], #PV becomes the unique elec source
    (213909,213913): [32179,32179*1.5],
            }


def modify_w_arrays(data_object, new_arrays):
    n = 1
    for k,v in new_arrays.items():
        n = len(v)        

    data, _ = data_object.get_resource(f"{data_object.metadata['name']}_technosphere_matrix.data")
    indices, _ = data_object.get_resource(f"{data_object.metadata['name']}_technosphere_matrix.indices")
    flip, _ = data_object.get_resource(f"{data_object.metadata['name']}_technosphere_matrix.flip")
    
    
    dp_dict = dict(
        zip(indices.astype("object"),[[tup[0], tup[1]] for tup in zip(data,flip)])
                        )
    for key, value in new_arrays.items():
        dp_dict[key][0] = value
    
    new_foreground = bwp.create_datapackage(
    fs = bwp.generic_zipfile_filesystem(dirpath=Path("./data/inputs"), filename=f"{data_object.metadata['name']}.zip", write=True),
    # combinatorial=True,
    sequential=True,
    )

    for row_col, data_flip in dp_dict.items():
        if not isinstance(data_flip[0], np.ndarray):
            new_array = np.full((n,), fill_value = data_flip[0])
            dp_dict[row_col][0] = new_array
            
    new_data = np.array([val[0] for val in dp_dict.items()])
                   
    new_foreground.add_persistent_array(
    matrix="technosphere_matrix",
    data_array=new_data,
    indices_array=indices.astype(bwp.INDICES_DTYPE),
    flip_array=flip,
    name=data_object.metadata['name'],
)

    # return dp_dict
    return new_foreground

In [ ]:
aa = modify_w_arrays(objects[2], new_arrays)

In [ ]:
for idx, col in scenarios_ids.iteritems():
    print(col)

In [ ]:
for idx, col in scenarios_ids.iteritems()

    lca_b = bc.LCA(
        demand=fu,
        data_objs=objects + [aa],
        use_distributions=False,
        use_arrays=True,
    #     seed_override=42,  # Seed should not be used
    )
    lca_b.lci()
    lca_b.lcia()
    lca_b.keep_first_iteration()
    iterations = 2
    scores_b = [lca_b.score for _ in zip(range(iterations), lca_b)]
    print("SCENARIO CALC", scores_b)